This script train a shallow FNN to approximated the discrete-time dynamics model of the lunar lander.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# read data

filepath = '/Data/'
DATA_dict = dict()
num_traj = 40
for i in range(num_traj):
    num = i+1;
    if num >= 10:
        str_num = str(num) # no leading zero
    else:
        str_num = '0'+str(num) # add leading zero
    filename = 'trajdata_'+str_num+'.csv'
    with open(filepath+filename, 'r') as f:
        # remove top row, and store the rest in dictionary
        DATA_dict[i] = np.genfromtxt(f, dtype='f4', delimiter=',', skip_header=1)

In [ ]:
# define the keras model
# 4-3-3-3-3-3-3-3-3-3 configuration, relu and softmax activation for the
# hidden layer and output layer respectively
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(3, input_dim=4, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'relu'),
    tf.keras.layers.Dense(3, activation='softmax')])

In [ ]:
# compile the keras model
net.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])